$\newcommand{\giraffe}{\texttt{GiRaFFE}}$
$\newcommand{\gf}{\texttt{GiRaFFEFood}}$
## $\gf$: An Einstein Toolkit Initial Data Thorn for $\giraffe$

### NRPy+ Source Code for this module: [GiRaFFEFood_HO.py](../edit/GiRaFFEFood_HO.py) , which is fully documented in the [previous NRPy+ tutorial module](Tutorial-GiRaFFEFood_HO.ipynb) on using NRPy+ to construct these Exact Wald initial data as SymPy expressions.


In this part of the tutorial, we will construct an Einstein Toolkit (ETK) thorn (module) that will set up *initial data* for $\giraffe$. In a [previous tutorial module](Tutorial-GiRaFFEFood_HO.ipynb), we used NRPy+ to contruct the SymPy expressions for Exact Wald initial data. 

We will construct this thorn in two steps.

1. Call on NRPy+ to convert the SymPy expressions for the initial data into one C-code kernel.
1. Write the C code and linkages to the Einstein Toolkit infrastructure (i.e., the .ccl files) to complete this Einstein Toolkit module.

This thorn requires Initial data to be set up in several stages.
1. Run the shifted Kerr-Schild thorn to set up the four-metric we wish to use.
1. (This module) Set up the four-vector potential $A_\mu$ (which includes $\Phi$ and $A_i$) and the Valencia 3-velocity.
1. Run the A-to-B driver from $\giraffe$ to fill in the initial $B^i$ data.
1. Run the Primitive-to-conservative solver to calculate $\tilde{S}_i$.

### Step 1: Call on NRPy+ to convert the SymPy expression for the Eact Wald initial data into a C-code kernel. 

After importing the core modules, we will set $\text{GridFuncMemAccess}$ to $\text{ETK}$. SymPy expressions for plane wave initial data are written inside [GiRaFFEFood_HO.py](../edit/GiRaFFEFood_HO.py), and we simply import them for use here.

In [1]:
# Step 1a: Import needed NRPy+ core modules:
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
from outputC import *
import loop


# Step 1b: This is an Einstein Toolkit (ETK) thorn. Here we
#          tell NRPy+ that gridfunction memory access will 
#          therefore be in the "ETK" style.
par.set_parval_from_str("grid::GridFuncMemAccess","ETK")
#Set the spatial dimension parameter to 3.
par.set_parval_from_str("grid::DIM", 3)
DIM = par.parval_from_str("grid::DIM")

# Step 1c: Within the ETK, the 3D gridfunctions x, y, and z store the
#          Cartesian grid coordinates. Setting the gri.xx[] arrays
#          to point to these gridfunctions forces NRPy+ to treat
#          the Cartesian coordinate gridfunctions properly --
#          reading them from memory as needed.
x,y,z = gri.register_gridfunctions("AUX",["x","y","z"])
gri.xx[0] = x
gri.xx[1] = y
gri.xx[2] = z

import GiRaFFEFood_HO.GiRaFFEFood_HO as gfho
gfho.GiRaFFEFood_HO()

# Step 2: Create the C code output kernel.
GiRaFFEFood_A_v_to_print = [\
                            lhrh(lhs=gri.gfaccess("out_gfs","AD0"),rhs=gfho.AD[0]),\
                            lhrh(lhs=gri.gfaccess("out_gfs","AD1"),rhs=gfho.AD[1]),\
                            lhrh(lhs=gri.gfaccess("out_gfs","AD2"),rhs=gfho.AD[2]),\
                            lhrh(lhs=gri.gfaccess("out_gfs","ValenciavU0"),rhs=gfho.ValenciavU[0]),\
                            lhrh(lhs=gri.gfaccess("out_gfs","ValenciavU1"),rhs=gfho.ValenciavU[1]),\
                            lhrh(lhs=gri.gfaccess("out_gfs","ValenciavU2"),rhs=gfho.ValenciavU[2]),\
                            ]

GiRaFFEFood_A_v_CKernel = fin.FD_outputC("returnstring",GiRaFFEFood_A_v_to_print,params="outCverbose=False")

# Format the code within a C loop over cctkGH
GiRaFFEFood_A_v_looped = loop.loop(["i2","i1","i0"],["0","0","0"],["cctk_lsh[2]","cctk_lsh[1]","cctk_lsh[0]"],\
                                   ["1","1","1"],["#pragma omp parallel for","",""],"",\
                                   GiRaFFEFood_A_v_CKernel.replace("time","cctk_time"))
# Step 3: Create directories for the thorn if they don't exist.
!mkdir GiRaFFEFood_HO     2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.
!mkdir GiRaFFEFood_HO/src 2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.

# Step 4: Write the C code kernel to file.
with open("GiRaFFEFood_HO/src/GiRaFFEFood_A_v_ExactWald.h", "w") as file:
    file.write(str(GiRaFFEFood_A_v_looped))


### Step 2: Interfacing with the Einstein Toolkit

#### Step 2a: Constructing the Einstein Toolkit C-code calling functions that include the C code kernels.

We will write another C file with the functions we need here.

In [2]:
%%writefile GiRaFFEFood_HO/src/InitialData.c
#include <math.h>
#include <stdio.h>

#include "cctk.h"
#include "cctk_Parameters.h"
#include "cctk_Arguments.h"
void GiRaFFE_set_A_v(const cGH* restrict const cctkGH,const int *cctk_lsh,const int *cctk_nghostzones,
                     const CCTK_REAL *xGF,const CCTK_REAL *yGF,const CCTK_REAL *zGF,const CCTK_REAL *u0GF,
                     CCTK_REAL *AD0GF,CCTK_REAL *AD1GF,CCTK_REAL *AD2GF,
                     CCTK_REAL *ValenciavU0GF,CCTK_REAL *ValenciavU1GF,CCTK_REAL *ValenciavU2GF) {

  DECLARE_CCTK_PARAMETERS;

#include "GiRaFFEFood_A_v_ExactWald.h"

}

void Write_to_HydroBase(const cGH* restrict const cctkGH,const int *cctk_lsh,const int *cctk_nghostzones,
                        const CCTK_REAL *ValenciavU0,const CCTK_REAL *ValenciavU1,const CCTK_REAL *ValenciavU2,
                        const CCTK_REAL *AD0,const CCTK_REAL *AD1,const CCTK_REAL *AD2,
                        CCTK_REAL *vel, CCTK_REAL *Avec) {
  /* Bvec[i] <- BUi
   * Avec[i] <- ADi
   * vel[i]  <- ValenciavUi
   */
  DECLARE_CCTK_PARAMETERS;
  
#pragma omp parallel for
  for(int i2=0; i2<cctk_lsh[2]; i2++) {
      for(int i1=0; i1<cctk_lsh[1]; i1++) {
          for(int i0=0; i0<cctk_lsh[0]; i0++) {
              CCTK_INT idx3;
              CCTK_INT idx4[3];
              idx3 = CCTK_GFINDEX3D(cctkGH, i0,i1,i2);
              idx4[0] = CCTK_GFINDEX4D(cctkGH, i0,i1,i2,0);
              idx4[1] = CCTK_GFINDEX4D(cctkGH, i0,i1,i2,1);
              idx4[2] = CCTK_GFINDEX4D(cctkGH, i0,i1,i2,2);
              Avec[idx4[0]] = AD0[idx3];
              Avec[idx4[1]] = AD1[idx3];
              Avec[idx4[2]] = AD2[idx3];
              vel[idx4[0]] = ValenciavU0[idx3];
              vel[idx4[1]] = ValenciavU1[idx3];
              vel[idx4[2]] = ValenciavU2[idx3];
              // We don't set Phi, because it is always set to zero in GiRaFFE ID thorns.
          }
      }
  }
}

void GiRaFFE_ExactWaldID(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;
  
  GiRaFFE_set_A_v(cctkGH,cctk_lsh,cctk_nghostzones,
                  x,y,z,u0,
                  AD0,AD1,AD2,
                  ValenciavU0,ValenciavU1,ValenciavU2);

  Write_to_HydroBase(cctkGH,cctk_lsh,cctk_nghostzones,
                     ValenciavU0,ValenciavU1,ValenciavU2,
                     AD0,AD1,AD2,
                     vel,Avec);
}

Overwriting GiRaFFEFood_HO/src/InitialData.c


### Step 2b: CCL files - Define how this module interacts and interfaces with the larger Einstein Toolkit infrastructure

Writing a module ("thorn") within the Einstein Toolkit requires that three "ccl" files be constructed, all in the root directory of the thorn:

1. $\text{interface.ccl}$: defines the gridfunction groups needed, and provides keywords denoting what this thorn provides and what it should inherit from other thorns. Specifically, this file governs the interaction between this thorn and others; more information can be found in the [official Einstein Toolkit documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-260000C2.2). 
With "implements", we give our thorn its unique name. By "inheriting" other thorns, we tell the Toolkit that we will rely on variables that exist and are declared "public" within those functions.

In [3]:
%%writefile GiRaFFEFood_HO/interface.ccl
implements: GiRaFFEFood_HO
inherits: admbase GiRaFFE_HO grid HydroBase ShiftedKerrSchild


Overwriting GiRaFFEFood_HO/interface.ccl


2. $\text{param.ccl}$: specifies free parameters within the thorn, enabling them to be set at runtime. It is required to provide allowed ranges and default values for each parameter. More information on this file's syntax can be found in the [official Einstein Toolkit documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-265000C2.3).

In [4]:
%%writefile GiRaFFEFood_HO/param.ccl
shares: grid

USES KEYWORD type

shares: GiRaFFE_HO
USES CCTK_REAL GAMMA_SPEED_LIMIT

restricted:
CCTK_KEYWORD initial_data "Type of initial data"
{
  "ExactWald"      :: "Exact Wald initial data"
} "ExactWald"

restricted:
CCTK_REAL M "Kerr-Schild BH mass. Probably should always set M=1."
{
  0.0:* :: "Must be positive"
} 1.0


Overwriting GiRaFFEFood_HO/param.ccl


3. $\text{schedule.ccl}$: allocates storage for gridfunctions, defines how the thorn's functions should be scheduled in a broader simulation, and specifies the regions of memory written to or read from gridfunctions. $\text{schedule.ccl}$'s official documentation may be found [here](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-268000C2.4). 

We specify here the standardized ETK "scheduling bins" in which we want each of our thorn's functions to run.

In [5]:
%%writefile GiRaFFEFood_HO/schedule.ccl
STORAGE: GiRaFFE_HO::GiRaFFE_vars[3]
STORAGE: GiRaFFE_HO::GiRaFFE_Vs[3]

STORAGE: HydroBase::rho[1],HydroBase::press[1],HydroBase::eps[1],HydroBase::vel[1],HydroBase::Avec[1],HydroBase::Aphi[1]
            
schedule GROUP GiRaFFE_Initial IN CCTK_INITIAL after HydroBase_Initial before GiRaFFE_ID_Converter
{
} "Schedule GiRaFFE functions in HydroBase_Initial"

schedule GiRaFFE_ExactWaldID in GiRaFFE_Initial as GiRaFFE_Food
{
  LANG: C
  READS: admbase::alp(Everywhere)
  READS: admbase::betax(Everywhere)
  READS: admbase::betay(Everywhere)
  READS: admbase::betaz(Everywhere)
  READS: admbase::gxx(Everywhere)
  READS: admbase::gxy(Everywhere)
  READS: admbase::gxz(Everywhere)
  READS: admbase::gyy(Everywhere)
  READS: admbase::gyz(Everywhere)
  READS: admbase::gzz(Everywhere)
  READS: grid::x(Everywhere)
  READS: grid::y(Everywhere)
  READS: grid::y(Everywhere)
  WRITES: GiRaFFE_HO::AD0(Everywhere)
  WRITES: GiRaFFE_HO::AD1(Everywhere)
  WRITES: GiRaFFE_HO::AD2(Everywhere)
  WRITES: GiRaFFE_HO::ValenciavU0(Everywhere)
  WRITES: GiRaFFE_HO::ValenciavU1(Everywhere)
  WRITES: GiRaFFE_HO::ValenciavU2(Everywhere)
} "Initial data for GiRaFFE"


Overwriting GiRaFFEFood_HO/schedule.ccl


#### Step 2c: Add the C code to the Einstein Toolkit compilation list.

We will also need $\text{make.code.defn}$, which indicates the list of files that need to be compiled. This thorn only has the one C file to compile.

In [6]:
%%writefile GiRaFFEFood_HO/src/make.code.defn
SRCS = InitialData.c

Overwriting GiRaFFEFood_HO/src/make.code.defn
